In [11]:
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt 
import netCDF4 as nc
from netCDF4 import Dataset
import pyproj
import pandas as pd

In [77]:
# Importing SINMOD data
filename_PhysStates = 'PhysStates.nc'

PhysStates_data = Dataset(filename_PhysStates, 'r')

# Extracting all the dimensions, we can extract both ways, I don't think it matters
xc = PhysStates_data.variables['xc'][:]  # x-coordinates (meters)
yc = PhysStates_data.variables['yc'][:]  # y-coordinates (meters)
zc = PhysStates_data['LayerDepths'][:]  # z-coordinates (meters)

# Extracting temperature at t = 0, and check the dimensions
temperature_var = PhysStates_data.variables['temperature']
temperature = temperature_var[0,:,:,:]

print(f"Temperature data shape: {temperature.shape}")
print(f"xc shape: {xc.shape}")
print(f"yc shape: {yc.shape}")
print(f"zc shape: {zc.shape}")

Temperature data shape: (25, 235, 300)
xc shape: (300,)
yc shape: (235,)
zc shape: (25,)


In [78]:
# Importing coral reef data

# gml_file_coral_reefs = "./KystOgFiskeri_50_Trondelag_25832_Korallrev_GML.gml"
gml_file_coral_reefs = "KystOgFiskeri_50_Trondelag_25833_Korallrev_GML.gml"

gdf_coral_reefs = gpd.read_file(gml_file_coral_reefs)

# Not sure this is necessary since all the naturtypeNavn are the same
# unique_naturtypeNavn = gdf_coral_reefs['naturtypeNavn'].unique()
# print(unique_naturtypeNavn)
coral_data = gdf_coral_reefs[gdf_coral_reefs['naturtypeNavn'] == 'Korallforekomster']

# I think it's already a dataframe, so also unnecessary
coral_df = pd.DataFrame(coral_data)


coral_location_df = coral_df[['lengdegrad', 'breddegrad']]

In [79]:
# Now to transform the coordinates to the same projection as the SINMOD data

from pyproj import CRS, Transformer

# Print grid mapping to see what the horizontal resolution is
# In the case of gin it is 20km, for nor4km it is 4km
grid_mapping = PhysStates_data.variables['grid_mapping']
print(grid_mapping)


<class 'netCDF4._netCDF4.Variable'>
int32 grid_mapping()
    grid_mapping_name: polar_stereographic
    straight_vertical_longitude_from_pole: 58.0
    horizontal_resolution: 20000.0
    latitude_of_projection_origin: 90.0
    longitude_of_projection_origin: 58.0
    standard_parallel: 60.0
    origoRef: [0. 0.]
    semi_minor_axis: 6370000.0
    semi_major_axis: 6370000.0
    false_easting: 3900000.0
    false_northing: 2570000.0
    scale_factor_at_projection_origin: 1.0
unlimited dimensions: 
current shape = ()
filling on, default _FillValue of -2147483647 used


In [87]:
# Specifying projection to SINMOD format

# Define the Coordinate Reference Systems (CRS) for the coral data
# It could be any of these, need to double check
crs_wgs84 = CRS.from_epsg(4326)  # WGS84 (lat/lon coordinates)
# crs_euref89_utm32 = CRS.from_epsg(25832)  # EUREF89 / UTM zone 32N
# crs_euref89 = CRS.from_epsg(4258)  # EUREF89 (geographic lat/lon)

# SINMOD projection parameters as a custom projection
crs_sinmod = CRS.from_proj4("+proj=stere +lat_0=90 +lat_ts=60 +lon_0=58 "
                            "+x_0=3900000 +y_0=2570000 +ellps=WGS84 +units=m +no_defs")

# Create a transformer to transform from wgs84 (geographic) to SINMOD
transformer_wgs84_to_sinmod = Transformer.from_crs(crs_wgs84, crs_sinmod, always_xy=True)

xc_grid, yc_grid = np.meshgrid(xc, yc)
lon_grid, lat_grid = transformer_wgs84_to_sinmod.transform(xc_grid, yc_grid)

# Performing projection on coral reef data
print("coral_lons range before:", coral_location_df['lengdegrad'].min(), coral_location_df['lengdegrad'].max())
print("coral_lats range before:", coral_location_df['breddegrad'].min(), coral_location_df['breddegrad'].max())
print("--------------------------------")

# Transform coral coordinates from EUREF89 to SINMOD
coral_x, coral_y = transformer_wgs84_to_sinmod.transform(coral_location_df['lengdegrad'], coral_location_df['breddegrad'])

# We need to DIVIDE by the resolution of the SINMOD grid to get the grid coordinates
# In the case of gin this is 20km
coral_x /= 20000
coral_y /= 20000

# The values after are NOT lat lon, but rather the SINMOD grid coordinates 
print("coral_lons grid-coordinates after:", coral_x.min(), coral_x.max())
print("coral_lats grid-coordinates before:", coral_y.min(), coral_y.max())

# So we expect them to be in the range of the SINMOD grid:
print(f"Grid dimensions (xc): {xc.shape[0]}")
print(f"Grid dimensions (yc): {yc.shape[0]}")


coral_lons range before: 7.84797 11.32001
coral_lats range before: 63.4133 65.00816
--------------------------------
coral_lons grid-coordinates after: 87.50081214535975 98.74281827314783
coral_lats grid-coordinates before: 33.6640977812021 39.715041231478516
Grid dimensions (xc): 300
Grid dimensions (yc): 235
